## Import library

In [1]:
import pandas as pd
import httpx
import json

## Set option pandas

In [2]:
pd.options.mode.use_inf_as_na = True
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Read configuration

In [3]:
with open("./credentials.json") as r:
    config = json.loads(r.read())
minio_config = dict(
    key=config["accessKey"],
    secret=config["secretKey"],
    client_kwargs={
        'endpoint_url': config["url"]
    }
)

with open("./api_config.json") as r:
    config = json.loads(r.read())
api_config = dict(
    url=config["url"],
    token=config["token"],
)

## Function call api

In [4]:
def get_process(httpx, url, headers, topic):
  url = url + f"?topic={topic}"
  with httpx.Client() as client:
    r = client.get(url=url, headers=headers)
    json = r.json()
  if json["ok"] == True and len(json["rows"]) != 0:
    return json["rows"]
  else:
    return {}


def set_process(httpx, url, headers, process_id, status):
  url = url + "/status"
  headers["Content-Type"] = "application/x-www-form-urlencoded"
  data = f"processId={process_id}&status={status}"
  with httpx.Client() as client:
    r = client.put(url=url, headers=headers, data=data)
    json = r.json()
  if (json["ok"] == True):
    return json
  else:
    return {}

## Get process from api

In [5]:
# topic = "service_opd_screening"
topic = "service_opd_screening"
url = api_config["url"]
token = api_config["token"]
headers = {"Authorization": "Bearer " + token}
r = get_process(httpx=httpx, url=url, headers=headers, topic=topic)

display(r)

{'process_id': 'service_opd_screening_1669361015383',
 'hospcode': '41124',
 'status': 'PROCESS',
 'topic': 'service_opd_screening',
 'path': 'upload/41124/service_opd_screening/service_opd_screening_1669361015383'}

## Set parameter after call api

In [6]:
hospcode = r["hospcode"]
process_id = r["process_id"]
source_path = "s3://mophgw/{}".format(r["path"])
destination_path = "s3://phr/{}".format(r["path"])
local_code_path = "s3://mapcode/{}/localcode".format(hospcode)
std_code_path = "s3://mapcode/{}/stdcode".format(hospcode)

## function to read_csv

In [7]:
def read_csv(path: str = "", filename: str = "", prefix_type: str = None):
  df = pd.read_csv(f"{path}/{filename}", engine="c",
              dtype="str", storage_options=minio_config)
  prefix = filename.removesuffix(".csv.gz")
  if prefix_type != "":
    df = df.add_prefix(add_prefix(prefix, prefix_type))
  return df


def add_prefix(name: str = None, prefix_type: str = None):
  if prefix_type == "local":
    return f"{name}_local"
  elif prefix_type == "std":
    return f"{name}_"
  else:
    return ""



## load transaction file to dataframe
- person

In [8]:
person = read_csv(path=source_path, filename="person.csv.gz")
person = person.drop_duplicates(subset=["cid"])

display(person)

,cid,hn,title_code,first_name,middle_name,last_name,birthdate,gender,rh_blood_group,blood_group,nationality_code,marital_status
0,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2
1,3320500591905,000121213,นาย,ระเบียบ,NaN,บุญงาม,1967-04-14T17:00:00.000Z,1,NaN,ไม่ทราบหมู่เลือด,99,2
2,5310400077840,000379109,นาง,ชำนาญ,NaN,เกิดสุข,1947-06-08T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2
3,1319800810485,000489688,ด.ญ.,พรรณวษา,NaN,นามไธสง,2018-03-16T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,1
4,3310401286519,000010933,นาง,สอาด,NaN,มีศรี,1949-12-31T17:00:00.000Z,2,NaN,AB,99,2
...,...,...,...,...,...,...,...,...,...,...,...,...
802,3310500132366,000401488,นาง,ทองสุข,NaN,ไทยแท้,1970-02-19T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2
803,5310600056579,000440033,นาย,ประสิทธิ์,NaN,คำพันธ์,1958-01-20T17:00:00.000Z,1,NaN,ไม่ทราบหมู่เลือด,99,2
804,3310400361878,000292481,นาย,ประมงค์,NaN,นิเวศกูล,1933-12-31T17:00:00.000Z,1,NaN,ไม่ทราบหมู่เลือด,99,1
805,3310400153167,000002614,นาง,สำรวย,NaN,สายโลหิต,1960-12-31T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2


## load localcode files to dataframe
- title, gender, rh_blood_group, blood_group, nationality, marital_status, occupation

In [9]:
local_config = ["blood_group", "gender", "marital_status",
                "nationality", "rh_blood_group", "title"]
local_df = dict()
for k in local_config:
  local_df[k] = read_csv(path=local_code_path, filename=k +
                         ".csv.gz", prefix_type="local")
  display(local_df[k])

,blood_group_localcode,blood_group_localname


,gender_localcode,gender_localname
0,2,หญิง
1,1,ชาย


,marital_status_localcode,marital_status_localname


,nationality_localcode,nationality_localname
0,95,"ไทย-จีน, จีน-ไทย(ยกเลิก)"
1,96,ไร้สัญชาติ
2,97,อื่น ๆ
3,94,"ไทย-อิสลาม, อิสลาม-ไทย(ยกเลิก)"
4,91,ผู้อพยพอินโดจีนสัญชาติกัมพูชา
...,...,...
102,81,ลาว(แรงงาน)
103,82,เซอร์เบียน
104,83,มอนเตเนกริน
105,989,บุคคลที่ไม่มีสถานะทางทะเบียน


,rh_blood_group_localcode,rh_blood_group_localname


,title_localcode,title_localname
0,Ms.,Ms.
1,Mrs.,Mrs.
2,Mr.,Mr.
3,Miss.,Miss.
4,อาสาสมัครทหารพราน,อาสาสมัครทหารพราน
...,...,...
200,จ.ท.,จ.ท.
201,จ.ต.หญิง,จ.ต.หญิง
202,จ.ต.,จ.ต.
203,คุณหญิง,คุณหญิง


## load stdcode file to dataframe
- title, gender, rh_blood_group, blood_group, nationality, marital_status, occupation

In [10]:
std_config = dict(
    blood_group=["blood_group", "blood_groups"],
    gender = ["gender","genders"], 
    marital_status = ["marital_status","marital_statuses"],
    nationality = ["nationality","nationalities"], 
    rh_blood_group = ["rh_blood_group","rh_blood_groups"], 
    title = ["title","titles"]
)
std_df = dict()
for k, v in std_config.items():
  std_df[k] = read_csv(path=std_code_path, filename=v[1] + ".csv.gz", prefix_type="std")
  display(std_df[k])

,blood_groups_localcode,blood_groups_stdcode,blood_groups_stdname_th,blood_groups_stdname_en


,genders_localcode,genders_stdcode,genders_stdname_th,genders_stdname_en
0,1,1,ชาย,Male
1,2,2,หญิง,Female
2,NaN,0,ไม่สามารถระบุได้,cannot be identified


,marital_statuses_localcode,marital_statuses_stdcode,marital_statuses_stdname_th,marital_statuses_stdname_en
0,1,1,โสด,Single
1,2,2,สมรส,Married
2,3,3,หม้าย,Widowed
3,4,4,หย่า,Divorced
4,5,5,แยกกันอยู่,Separated
...,...,...,...,...
774,NaN,9005,ลูกจ้างชั่วคราว,NaN
775,NaN,9006,เจ้าพนักงานทันตสาธารณสุข,NaN
776,NaN,9007,เจ้าพนักงานสาธารณสุข,NaN
777,NaN,9998,ไม่ทราบ / ไม่ระบุ,NaN


,nationalities_localcode,nationalities_stdcode,nationalities_stdname_th,nationalities_stdname_en
0,02,2,โปรตุเกส,PORTUGUESE
1,03,3,ดัตช์,NETHERLANDS
2,04,4,เยอรมัน,W. GERMANY
3,05,5,ฝรั่งเศส,FRANCE
4,06,6,เดนมาร์ก,DENMARK
...,...,...,...,...
499,NaN,NaN,NaN,NaN
500,NaN,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN
502,NaN,NaN,NaN,NaN


,rh_blood_groups_localcode,rh_blood_groups_stdcode,rh_blood_groups_stdname_th,rh_blood_groups_stdname_en


,titles_localcode,titles_stdcode,titles_stdname_th,titles_stdname_en
0,NaN,001,เด็กชาย,NaN
1,NaN,002,เด็กหญิง,NaN
2,NaN,003,นาย,MR.
3,NaN,004,นางสาว,MISS
4,NaN,005,นาง,MRS.
...,...,...,...,...
499,NaN,956,พระครูสมุทรขันตยาภรณ์,NaN
500,NaN,957,พระครูภาวนาวรกิจ,NaN
501,NaN,958,พระครูศรีศาสนคุณ,NaN
502,NaN,959,พระครูวิบูลย์ธรรมศาสก์,NaN


## merge localcode to person

In [11]:
def merge_codes(main_df: pd.DataFrame, pathname: str, prefix_type: str, merge_config: dict) -> pd.DataFrame :
  main_cols = main_df.columns
  for k, v in merge_config.items():
    if v[0] in main_cols:
      ldf = read_csv(pathname, k + ".csv.gz", prefix_type)
      main_df = pd.merge(main_df,ldf, how = "left", left_on = v[0], right_on = add_prefix(k,prefix_type) + v[1])
  return main_df


In [12]:
person_merge_local = dict(
    title=["title_code", "code"],
    gender=["gender", "code"],
    rh_blood_group=["rh_blood_group", "code"],
    blood_group=["blood_group", "code"],
    nationality=["nationality_code", "code"],
    marital_status=["marital_status", "code"],
)
person = merge_codes(person, local_code_path, prefix_type="local", merge_config=person_merge_local)
      
# person = person.drop_duplicates(subset=["cid"])

display(person)

,cid,hn,title_code,first_name,middle_name,last_name,birthdate,gender,rh_blood_group,blood_group,nationality_code,marital_status,title_localcode,title_localname,gender_localcode,gender_localname,rh_blood_group_localcode,rh_blood_group_localname,blood_group_localcode,blood_group_localname,nationality_localcode,nationality_localname,marital_status_localcode,marital_status_localname
0,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
1,3320500591905,000121213,นาย,ระเบียบ,NaN,บุญงาม,1967-04-14T17:00:00.000Z,1,NaN,ไม่ทราบหมู่เลือด,99,2,นาย,นาย,1,ชาย,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
2,5310400077840,000379109,นาง,ชำนาญ,NaN,เกิดสุข,1947-06-08T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
3,1319800810485,000489688,ด.ญ.,พรรณวษา,NaN,นามไธสง,2018-03-16T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,1,ด.ญ.,ด.ญ.,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
4,3310401286519,000010933,นาง,สอาด,NaN,มีศรี,1949-12-31T17:00:00.000Z,2,NaN,AB,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,3310500132366,000401488,นาง,ทองสุข,NaN,ไทยแท้,1970-02-19T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
772,5310600056579,000440033,นาย,ประสิทธิ์,NaN,คำพันธ์,1958-01-20T17:00:00.000Z,1,NaN,ไม่ทราบหมู่เลือด,99,2,นาย,นาย,1,ชาย,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
773,3310400361878,000292481,นาย,ประมงค์,NaN,นิเวศกูล,1933-12-31T17:00:00.000Z,1,NaN,ไม่ทราบหมู่เลือด,99,1,นาย,นาย,1,ชาย,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
774,3310400153167,000002614,นาง,สำรวย,NaN,สายโลหิต,1960-12-31T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN


## merge stdcode to person

In [13]:
person_merge_std = dict(
    titles=["title_localcode", "localcode"],
    genders=["gender_localcode", "localcode"],
    rh_blood_groups=["rh_blood_group_localcode", "localcode"],
    blood_groups=["blood_group_localcode", "localcode"],
    nationalities=["nationality_localcode", "localcode"],
    marital_statuses=["marital_status_localcode", "localcode"],
)
person = merge_codes(person, std_code_path,
                     prefix_type="std", merge_config=person_merge_std)
# person = person.drop_duplicates(subset=["cid"])

display(person)

,cid,hn,title_code,first_name,middle_name,last_name,birthdate,gender,rh_blood_group,blood_group,nationality_code,marital_status,title_localcode,title_localname,gender_localcode,gender_localname,rh_blood_group_localcode,rh_blood_group_localname,blood_group_localcode,blood_group_localname,nationality_localcode,nationality_localname,marital_status_localcode,marital_status_localname,titles_localcode,titles_stdcode,titles_stdname_th,titles_stdname_en,genders_localcode,genders_stdcode,genders_stdname_th,genders_stdname_en,rh_blood_groups_localcode,rh_blood_groups_stdcode,rh_blood_groups_stdname_th,rh_blood_groups_stdname_en,blood_groups_localcode,blood_groups_stdcode,blood_groups_stdname_th,blood_groups_stdname_en,nationalities_localcode,nationalities_stdcode,nationalities_stdname_th,nationalities_stdname_en,marital_statuses_localcode,marital_statuses_stdcode,marital_statuses_stdname_th,marital_statuses_stdname_en
0,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,9,อื่นๆ,Other
1,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,NaN,NaN,NaN
2,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,NaN,NaN,NaN
3,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,NaN,NaN,NaN
4,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955246,3310400843848,000289062,นาง,บันหยัด,NaN,ราชาทรัพย์,1946-12-31T17:00:00.000Z,2,NaN,AB,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,9004,ลูกจ้างประจำ,NaN
2955247,3310400843848,000289062,นาง,บันหยัด,NaN,ราชาทรัพย์,1946-12-31T17:00:00.000Z,2,NaN,AB,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,9005,ลูกจ้างชั่วคราว,NaN
2955248,3310400843848,000289062,นาง,บันหยัด,NaN,ราชาทรัพย์,1946-12-31T17:00:00.000Z,2,NaN,AB,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,9006,เจ้าพนักงานทันตสาธารณสุข,NaN
2955249,3310400843848,000289062,นาง,บันหยัด,NaN,ราชาทรัพย์,1946-12-31T17:00:00.000Z,2,NaN,AB,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,9007,เจ้าพนักงานสาธารณสุข,NaN


## rename columns

In [14]:
cols_rename = dict(
    group = "groups",
    nationality="nationalities",
    status = "statuses",
    gender = "genders",
    title = "titles"
)
def rename_columns(df: pd.DataFrame, cols_rename: dict[str,str])->pd.DataFrame:        
    cols = df.columns.to_list()
    for k,v in cols_rename.items():
        for i, col in enumerate(cols):
            if v in col:
                cols[i] = col.replace(v,k)
    df.columns = cols
    return df

person = rename_columns(person,cols_rename)
display(person)

Index(['cid', 'hn', 'title_code', 'first_name', 'middle_name', 'last_name', 'birthdate', 'gender', 'rh_blood_group', 'blood_group', 'nationality_code', 'marital_status', 'title_localcode', 'title_localname', 'gender_localcode', 'gender_localname', 'rh_blood_group_localcode', 'rh_blood_group_localname', 'blood_group_localcode', 'blood_group_localname', 'nationality_localcode', 'nationality_localname', 'marital_status_localcode', 'marital_status_localname', 'titles_localcode', 'titles_stdcode', 'titles_stdname_th', 'titles_stdname_en', 'genders_localcode', 'genders_stdcode', 'genders_stdname_th', 'genders_stdname_en', 'rh_blood_groups_localcode', 'rh_blood_groups_stdcode', 'rh_blood_groups_stdname_th', 'rh_blood_groups_stdname_en', 'blood_groups_localcode', 'blood_groups_stdcode', 'blood_groups_stdname_th', 'blood_groups_stdname_en', 'nationalities_localcode', 'nationalities_stdcode', 'nationalities_stdname_th', 'nationalities_stdname_en', 'marital_statuses_localcode',
       'marital_st

,cid,hn,title_code,first_name,middle_name,last_name,birthdate,gender,rh_blood_group,blood_group,nationality_code,marital_status,title_localcode,title_localname,gender_localcode,gender_localname,rh_blood_group_localcode,rh_blood_group_localname,blood_group_localcode,blood_group_localname,nationality_localcode,nationality_localname,marital_status_localcode,marital_status_localname,title_localcode,title_stdcode,title_stdname_th,title_stdname_en,gender_localcode,gender_stdcode,gender_stdname_th,gender_stdname_en,rh_blood_group_localcode,rh_blood_group_stdcode,rh_blood_group_stdname_th,rh_blood_group_stdname_en,blood_group_localcode,blood_group_stdcode,blood_group_stdname_th,blood_group_stdname_en,nationality_localcode,nationality_stdcode,nationality_stdname_th,nationality_stdname_en,marital_status_localcode,marital_status_stdcode,marital_status_stdname_th,marital_status_stdname_en
0,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,9,อื่นๆ,Other
1,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,NaN,NaN,NaN
2,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,NaN,NaN,NaN
3,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,NaN,NaN,NaN
4,3320800148446,000330434,นาง,กุลธิดา,NaN,นามวงศา,1983-12-13T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955246,3310400843848,000289062,นาง,บันหยัด,NaN,ราชาทรัพย์,1946-12-31T17:00:00.000Z,2,NaN,AB,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,9004,ลูกจ้างประจำ,NaN
2955247,3310400843848,000289062,นาง,บันหยัด,NaN,ราชาทรัพย์,1946-12-31T17:00:00.000Z,2,NaN,AB,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,9005,ลูกจ้างชั่วคราว,NaN
2955248,3310400843848,000289062,นาง,บันหยัด,NaN,ราชาทรัพย์,1946-12-31T17:00:00.000Z,2,NaN,AB,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,9006,เจ้าพนักงานทันตสาธารณสุข,NaN
2955249,3310400843848,000289062,นาง,บันหยัด,NaN,ราชาทรัพย์,1946-12-31T17:00:00.000Z,2,NaN,AB,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN,NaN,NaN,NaN,NaN,2,2,หญิง,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,99,ไทย,THAILAND,NaN,9007,เจ้าพนักงานสาธารณสุข,NaN


In [15]:
# with open("./person_config.json", "w") as fw:
#   fw.write(json.dumps(dict(std=person_merge_std, local=person_merge_local)))